In [ ]:
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import scipy.ndimage.filters
from matplotlib.colors import LinearSegmentedColormap, rgb_to_hsv, hsv_to_rgb
import polyline
import folium
import requests
from urllib.parse import urlencode, quote
import json
from os import path
import math
import io
import datetime
import matplotlib.pyplot as plt
from shapely.geometry import Point
pd.options.mode.chained_assignment = None 
import geopandas as gpd

In [ ]:
df = pd.read_csv('data/OD_2019-07.csv')
st = pd.read_csv('data/Stations_2019.csv')

In [ ]:
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

In [ ]:
df['hour'] = df['start_date'].map(lambda x : x.hour)
df['dow'] = df['start_date'].map(lambda x : x.weekday())

In [ ]:
data = df.merge(st, left_on = 'start_station_code', right_on = 'Code')
data.rename(columns=dict(zip(st.columns.values,["start_station_" + x.lower() for x in st.columns.values])),inplace = True)

data = data.merge(st, left_on = 'end_station_code', right_on = 'Code')
data.rename(columns=dict(zip(st.columns.values,["end_station_" + x.lower() for x in st.columns.values])),inplace = True)
data = data.loc[:,~data.columns.duplicated()]

In [ ]:
def get_directions(c1,c2,s1,s2):
    coord = quote("%f,%f;%f,%f" % (c1[1],c1[0],c2[1],c2[0]))
    url = 'https://api.mapbox.com/directions/v5/mapbox/cycling/' 
    params = {
        'access_token': token,
        'steps': 'false',
        'alternatives': 'false',
        'geometries': 'polyline'
    }
    r = requests.get(url+coord,params=params)
    with open('directions/%s_%s.json' % (s1,s2), 'w') as outfile:
        json.dump(r.json(), outfile)
    
    return r.json()['routes'][0]['geometry']

In [ ]:
token = '###'

In [ ]:
data = data.loc[:,~data.columns.duplicated()]
comb = data.groupby(['start_station_code',
              'start_station_latitude', 
              'start_station_longitude', 
              'end_station_code',
              'end_station_latitude', 
              'end_station_longitude']
            ).size().to_frame('count').reset_index().sort_values('count', ascending = False)

In [ ]:
routes = comb[(comb.start_station_code != comb.end_station_code)].reset_index(drop=True)

In [ ]:
for i, row in routes.iloc[0:30000].iterrows():
    start_coord = (row['start_station_latitude'], row['start_station_longitude'])
    end_coord = (row['end_station_latitude'],row['end_station_longitude'])
    if not path.exists('directions/%s_%s.json' % (str(int(row['start_station_code'])),str(int(row['end_station_code'])))):
        geom = get_directions(start_coord,
                              end_coord,
                              str(int(row['start_station_code'])),
                              str(int(row['end_station_code']))
                             )

In [ ]:
def get_json_geometry(row):
    s1 = str(int(row['start_station_code']))
    s2 = str(int(row['end_station_code']))
    fp = 'directions/%s_%s.json' % (s1,s2)
    if not path.exists(fp):
        start_coord = (row['start_station_latitude'], row['start_station_longitude'])
        end_coord = (row['end_station_latitude'],row['end_station_longitude'])
        get_directions(start_coord,end_coord,s1,s2)                                      
    with open(fp) as f:
        data = json.load(f)
    return data['routes'][0]['geometry']

In [ ]:
def get_polyline_length(coord):
    length = 0
    for i in range(len(coord)-1):
        length += get_distance(coord[i], coord[i+1])
    return length

def get_distance(xy1,xy2):
    return math.sqrt((xy2[0]-xy1[0])**2 + (xy2[1]-xy1[1])**2)

def get_waypoint(coord,pct):
    
    tgt_dis = get_polyline_length(coord) * pct

    tot_dis = 0
    for i in range(len(coord)-1):
        dis = get_distance(coord[i], coord[i+1])
        tot_dis += dis
        if tot_dis >= tgt_dis:
            x1,y1 = coord[i]
            x2,y2 = coord[i+1]
            a = x2-x1
            b = y2-y1
            
            
            if a == 0 or b == 0:
                waypoint = (x1+a,y1+b)
                path = coord[:i+1]
                break
                
            a = abs(a)
            b = abs(b)
            
            c = math.sqrt(a**2 + b**2)

            C = math.pi/2
            
            B = math.acos((a**2+c**2-b**2)/(2*a*c))
            A = math.pi-B-C

            X = A
            Y = B
            Z = C

            z = dis - (tot_dis - tgt_dis)

            x = z*math.sin(X)/math.sin(Z)
            y = z*math.sin(Y)/math.sin(Z)
            
            x_fac = (x2-x1)/abs(x2-x1) if abs(x2-x1) > 0 else 1
            y_fac = (y2-y1)/abs(y2-y1) if abs(y2-y1) > 0 else 1
            
            waypoint = (x1+x*x_fac,y1+y*y_fac)
            path = coord[:i+1]
            break
            
    path.append(waypoint)

    my_dict = dict({"waypoint" : waypoint, "path":path})
    return my_dict

In [ ]:
def get_frame(bike_data, center, timestamp):

    m = folium.Map(
        location = center,
        zoom_start = 12,
        tiles = "CartoDB dark_matter"
    )

    for i, row in bike_data.iterrows():
        if row['geometry'] is not None:
            coord = polyline.decode(row['geometry'])
            dur = (row['end_date'] - row['start_date']).seconds
            ela = (timestamp - row['start_date']).seconds
            factor = float(ela/dur)
            xys = get_waypoint(coord,factor)['path']
            folium.PolyLine(
                xys,
                opacity = 0.7, 
                smoothFactor = 3,
                weight = 1,
            ).add_to(m)

    for i, row in bike_data.iterrows():
        if row['geometry'] is not None:
            coord = polyline.decode(row['geometry'])
            dur = (row['end_date'] - row['start_date']).seconds
            ela = (timestamp - row['start_date']).seconds
            factor = float(ela/dur)
            x,y = get_waypoint(coord,factor)['waypoint']

            folium.Circle([x,y], radius = 0.01, color = 'white' ,opacity = 1,).add_to(m)
    
    im = Image.open(io.BytesIO(m._to_png()))
    draw = ImageDraw.Draw(im)
    font = ImageFont.truetype('Roboto-BoldCondensed.ttf',25)
    draw.text((50, 125),str(timestamp),(255,255,255),font=font)
    
    return im

In [ ]:
start = datetime.datetime.now()

start_datetime = datetime.datetime(2019,7,22,7,59,50)
end_datetime = datetime.datetime(2019,7,22,9,30,0)

delta = datetime.timedelta(seconds = 10)
cnt = (end_datetime-start_datetime)/delta

img = []
while start_datetime < end_datetime:
    current_datetime = start_datetime + delta
    curr = cnt - (end_datetime-current_datetime)/delta
    print(f"Processing frame {str(int(curr))} of {str(int(cnt))} ({str(int(curr/cnt*100))}%)")
    subset = data[(data.start_date <= start_datetime) & (data.end_date > current_datetime) & (data.start_station_code != data.end_station_code)]
    subset['geometry'] = subset.apply(lambda row: get_json_geometry(row), axis=1)
    center = [data.start_station_latitude.mean(),data.start_station_longitude.mean()]
    im = get_frame(subset, center, current_datetime)
    img.append(im)
    start_datetime = current_datetime

In [ ]:
img_crop = []
for i in img:
    width, height = i.size 
    img_crop.append(i.crop((0,75,width-200,height-20)))
    
img_crop[0].save('bixi.gif',save_all=True,append_images=img_crop[1:], duration = 40,loop=0)